# Install

In [3]:
install.packages("remotes")
require(remotes)
remotes::install_github('JEFworks-Lab/STdeconvolve')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: remotes


vctrs        (NA    -> 0.6.0     ) [CRAN]
stringi      (NA    -> 1.7.12    ) [CRAN]
rlang        (NA    -> 1.1.0     ) [CRAN]
magrittr     (NA    -> 2.0.3     ) [CRAN]
lifecycle    (NA    -> 1.0.3     ) [CRAN]
glue         (NA    -> 1.6.2     ) [CRAN]
cli          (NA    -> 3.6.0     ) [CRAN]
Rcpp         (1.0.1 -> 1.0.10    ) [CRAN]
gtable       (NA    -> 0.3.1     ) [CRAN]
utf8         (NA    -> 1.2.3     ) [CRAN]
withr        (NA    -> 2.5.0     ) [CRAN]
pkgconfig    (NA    -> 2.0.3     ) [CRAN]
fansi        (NA    -> 1.0.4     ) [CRAN]
R6           (NA    -> 2.5.1     ) [CRAN]
pillar       (NA    -> 1.8.1     ) [CRAN]
generics     (NA    -> 0.1.3     ) [CRAN]
cpp11        (NA    -> 0.4.3     ) [CRAN]
tidyselect   (NA    -> 1.2.0     ) [CRAN]
tibble       (NA    -> 3.2.0     ) [CRAN]
stringr      (NA    -> 1.5.0     ) [CRAN]
purrr        (NA    -> 1.0.1     ) [CRAN]
dplyr        (NA    -> 1.1.0     ) [CRAN]
colorspace   (NA    -> 2.1-0     ) [CRAN]
lattice      (NA    -> 0.20-45   )

Installing 66 packages: vctrs, stringi, rlang, magrittr, lifecycle, glue, cli, Rcpp, gtable, utf8, withr, pkgconfig, fansi, R6, pillar, generics, cpp11, tidyselect, tibble, stringr, purrr, dplyr, colorspace, lattice, viridisLite, RColorBrewer, munsell, labeling, farver, Matrix, nlme, scales, mgcv, MASS, isoband, ggplot2, RcppEigen, systemfonts, polyclip, tweenr, formatR, futile.options, lambda.r, BH, xml2, slam, NLP, plyr, RcppArmadillo, matrixStats, cluster, gridExtra, tidyr, ggfun, ggforce, snow, futile.logger, tm, modeltools, reshape2, liger, clue, viridis, scatterpie, BiocParallel, topicmodels
Warning message in i.p(...):
“installation of package ‘stringi’ had non-zero exit status”Warning message in i.p(...):
“installation of package ‘rlang’ had non-zero exit status”Warning message in i.p(...):
“installation of package ‘magrittr’ had non-zero exit status”Warning message in i.p(...):
“installation of package ‘glue’ had non-zero exit status”Warning message in i.p(...):
“installation 

* checking for file ‘/private/var/folders/jw/qpdw77lx345f6f7h92xk197m6h04p_/T/Rtmptcu1EM/remotes7ed11aa9b4a9/JEFworks-Lab-STdeconvolve-1eb0de6/DESCRIPTION’ ... OK
* preparing ‘STdeconvolve’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
* building ‘STdeconvolve_1.3.1.tar.gz’


Warning message in i.p(...):
“installation of package ‘/var/folders/jw/qpdw77lx345f6f7h92xk197m6h04p_/T//Rtmptcu1EM/file7ed174b3fa66/STdeconvolve_1.3.1.tar.gz’ had non-zero exit status”

In [5]:
remotes::install_version("Seurat", version = "4.3.0")

Warning message:
“package ‘Seurat’ is not available (for R version 3.6.0)”

In [4]:
library(tidyverse)
library(Seurat)
library(STdeconvolve)

ERROR: Error in library(tidyverse): there is no package called ‘tidyverse’


# Functions

In [ ]:
marker_gene_list<-function(topic=1,exp_value=2,Gexp){
  ## highly expressed in cell-type of interest
  highgexp <- names(which(Gexp[topic,] > exp_value))
  ## high log2(fold-change) compared to other deconvolved cell-types
  log2fc <- sort(log2(Gexp[topic,highgexp]/colMeans(Gexp[-topic,highgexp])), decreasing=TRUE)
  return(tibble(Gene=names(log2fc),log2fc=log2fc))
}

In [ ]:
plot_spatial<-function(plot_data=plot_data,suffix1='_prop.eps',suffix2='_prop_lim1.eps',y_max=1,dir=dir,i){
  p1<-ggplot(plot_data, aes(x, y,fill = prop)) +
    geom_point(shape=21,size=4) + 
    guides(size="none")+
    labs(title=str_c("topic ",i))+
    scale_fill_viridis_c() +
    theme_void()+
    coord_equal()
  
  p2<-ggplot(plot_data, aes(x, y,fill = prop)) +
    geom_point(shape=21,size=4) + 
    guides(size="none")+
    labs(title=str_c("topic ",i))+
    scale_fill_viridis_c(limits=c(0,y_max)) +
    theme_void()+
    coord_equal()
  ggsave(plot = p1, paste(dir,"topic_",i,suffix1, sep=''), 
         height=5, width=5, units='in', dpi=300)
  ggsave(plot= p2, paste(dir,"topic_",i,suffix2, sep=''), 
         height=5, width=5, units='in', dpi=300)
}

In [ ]:
run_me_results<-function(opt,
                         dir ){
  optLDA <- optimalModel(models = ldas, opt = opt)
  results <- getBetaTheta(optLDA,
                          perc.filt = 0.05,
                          betaScale = 1000)
  deconProp <- results$theta
  deconGexp <- results$beta
  
  dir.create(dir)
  
  for(i in 1:dim(deconProp)[2]){
    plot_data<-merge(pos,deconProp[,i],by = 0)
    names(plot_data)<-c("barcode","x","y","prop")
    
    plot_spatial(plot_data=plot_data,suffix1='_prop.eps',suffix2='_prop.jpg',y_max=1,dir=dir,i=i)
    
  }

In [ ]:
marker_gene_output<-map(.x = 1:dim(deconGexp)[1],
                          ~marker_gene_list(topic = .x,exp_value = 2,Gexp = deconGexp))
  names(marker_gene_output)<-str_c("topic_genes_exp2.",1:dim(deconGexp)[1],".csv")
  
  map2(.x = names(marker_gene_output),
       .y = marker_gene_output,
       ~write_csv(x = .y,file = paste(dir,.x)))
  
  alpha<-map(.x = 1:21,~ldas$model[[.x]]@alpha)%>%unlist
  
  plot_df<-tibble(K=2:22,alpha=alpha,perplexities=ldas$perplexities,rare=ldas$numRare)
  plot_df
  
  p1<-ggplot(data = plot_df) +
    geom_line(mapping = aes(x = K,y = perplexities), color="red3",size=2) +
    geom_point(mapping = aes(x = K,y = perplexities),shape=21, color="black", fill=ifelse(alpha > 1, "white", "red3"), size=6)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("perplexity")+
    ylim(min(plot_df$perplexities)-10,10+max(plot_df$perplexities))
  p2<-ggplot(data = plot_df) +
    geom_point(mapping = aes(x = K,y = rare),shape=21, color="black", fill="blue", size=4)+ 
    geom_line(mapping = aes(x = K,y = rare), color="blue",size=2)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("cell−types with mean proportion < 5%")
  
  p3<-ggplot(data = plot_df) +
    geom_line(mapping = aes(x = K,y = alpha), color="darkgreen",size=2) +ylim(c(0,1))+
    geom_point(mapping = aes(x = K,y = alpha),shape=21, color="black", fill=ifelse(alpha > 1, "white", "darkgreen"), size=6)+
    theme_linedraw(base_size = 16,base_rect_size =2,base_line_size = 2)+ylab("alpha")
  
  print(p1+p2+p3)
  
  ggsave(plot = p1+p2+p3,filename =str_c(dir,"merged_QC_plot.eps") ,height=5, width=12, units='in', dpi=300)
  ggsave(plot = p1+p2+p3,filename = str_c(dir,"merged_QC_plot.jpg"),height=5, width=12, units='in', dpi=300)
  
}

# load data and preprocess data

Note: check R_input_files folder for input data

In [ ]:
counts<-Read10X_h5(filename = "Data/h5/VisiumFFPE_Mouse_Brain_Transgenic_Age_17p9_Rep_1.h5")
counts

In [ ]:
spatial_barcodes<-read_csv("Annotation_Spatial/spatial_cord_subset_17p9_rep1.csv")

colnames(counts)%in%spatial_barcodes$barcode%>%summary

In [ ]:
counts_subset <- counts[,colnames(counts)%in%spatial_barcodes$barcode]

pos<-as.data.frame(spatial_barcodes)
rownames(pos)<-pos[,1]

In [ ]:
pos<-pos[,5:6]
names(pos)<-c("x","y")
head(pos)

In [ ]:
counts_subset_clean <- cleanCounts(counts = counts_subset,
                                   min.lib.size = 100,
                                   min.reads = 1,
                                   min.detected = 1,
                                   verbose = TRUE)
## feature select for genes
##I am going to use the over dispersed genes and an astrocyte subset.
odGenes <- getOverdispersedGenes(as.matrix(counts_subset_clean),
                                 gam.k=5,
                                 alpha=0.05,
                                 plot=FALSE,
                                 use.unadjusted.pvals=FALSE,
                                 do.par=TRUE,
                                 max.adjusted.variance=1e3,
                                 min.adjusted.variance=1e-3,
                                 verbose=FALSE, details=TRUE)

In [ ]:
genes <- odGenes$ods
length(genes)

In [ ]:
astro<-read_csv(file = "Data/astro_markers.csv")
astro_overlap_fit<-rownames(counts_subset_clean)%in%astro$`Astrocyte Markers`
astro_overlap<-rownames(counts_subset_clean)[astro_overlap_fit]
gene_astro<-c(genes,astro_overlap)%>%unique()

In [ ]:
corpus<-preprocess(t(as.matrix(counts_subset_clean)),
                   selected.genes = gene_astro,plot=FALSE,
                   min.reads = 1, 
                   min.lib.size = 100, 
                   min.detected = 1,
                   ODgenes = FALSE, 
                   verbose = TRUE)

In [ ]:
ldas <- fitLDA(corpus$corpus, Ks = seq(2, 22, by = 1),
               perc.rare.thresh = 0.05,
               plot=FALSE,
               verbose=TRUE)


In [ ]:
saveRDS(object = ldas,file = "optlDA.17p9_rep1_astrogenes.rds")
#ldas<-readRDS(file = "optlDA.17p9_rep1_astrogenes.rds")

In [ ]:
run_me_results(opt=18,dir = "output_18_Tran_17p9_rep1_astrogenes_astro_newplots/")
#run_me_results(opt=12,dir = "output_12_Tran_17p9_rep1_astrogenes_astro/")